## Import libraries

In [1]:
import sys
import os

# Adiciona a pasta raiz do projeto ao sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import src.config as config


2025-07-11 20:15:42.132 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\time-series-prediction-rnn


## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [2]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 9
    # -----------------------------------------
    )

2025-07-11 20:15:45.881 | INFO     | src.dataset:main:33 - Starting raw data loading...
2025-07-11 20:15:45.881 | INFO     | src.dataset:main:39 - Requesting information between 2016-07-13 and 2025-07-11
2025-07-11 20:15:45.895 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading IndBovespa (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-07-11 20:15:49.005 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading IndNasdaq (^IXIC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-07-11 20:15:51.736 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading IndSp500 (^GSPC) from yfinance...


...

2025-07-11 20:16:46.986 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-07-11 20:17:05.283 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-07-11 20:17:26.378 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-07-11 20:17:44.762 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-07-11 20:18:14.944 | WARNING  | src.utils.dataset.dataset_loading_strategy:_request_bcb_series:118 - Erro ao conectar à API do BCB: Expecting value: line 1 column 1 (char 0)
2025-07-11 20:18:14.945 | WARNING  | src.utils.dataset.dataset_loading_strategy:_load_single_ticker:94 - No data returned for SELIC_Meta_Anual (432)
2025-07-11 20:18:16.950 | INFO     | src.utils

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')


            Close_^BVSP  High_^BVSP  Low_^BVSP  Open_^BVSP  Volume_^BVSP  \
2025-07-05          NaN         NaN        NaN         NaN           NaN   
2025-07-06          NaN         NaN        NaN         NaN           NaN   
2025-07-07     139490.0    141342.0   139295.0    141265.0     6117300.0   
2025-07-08     139303.0    139591.0   138770.0    139491.0     6750100.0   
2025-07-09     137481.0    139331.0   137299.0    139303.0     7584500.0   
2025-07-10     136743.0    137472.0   136015.0    137472.0     9566200.0   
2025-07-11          NaN         NaN        NaN         NaN           NaN   

             Close_^IXIC    High_^IXIC     Low_^IXIC    Open_^IXIC  \
2025-07-05           NaN           NaN           NaN           NaN   
2025-07-06           NaN           NaN           NaN           NaN   
2025-07-07  20412.519531  20511.820312  20323.019531  20490.550781   
2025-07-08  20418.460938  20480.890625  20377.359375  20466.929688   
2025-07-09  20611.339844  20645.410156  2

c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
INFO:src.utils.dataset.clean_strategy:Columns removed by f_classif: ['Volume_HO=F']
c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:src.utils.dataset.clean_strategy:Columns removed by f_regression: ['IGP_M_Mensal']


            Close_^BVSP  Volume_^BVSP  Volume_^GSPC  Volume_BTC-USD  \
2025-07-02    -0.003569      0.388641     -0.100330        0.275171   
2025-07-03     0.013499     -0.309774     -0.401650       -0.102294   
2025-07-04     0.003903     -0.652379      0.000000       -0.156022   
2025-07-07    -0.014052      1.893298      0.550198        0.065685   
2025-07-08    -0.001341      0.103444      0.095917       -0.024958   
2025-07-09    -0.013079      0.123613     -0.161297        0.308142   
2025-07-10    -0.005368      0.261283      0.115099        0.655720   

            Volume_GC=F  Volume_CL=F  IPCA_Mensal  IGP_M_Mensal month_name  \
2025-07-02    -0.894879     0.372941          0.0           0.0       July   
2025-07-03    -1.000000    -1.000000          0.0           0.0       July   
2025-07-04     0.000000     0.000000          0.0           0.0       July   
2025-07-07     0.000000     0.000000          0.0           0.0       July   
2025-07-08     2.578740    -0.339593     

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [3]:
from src import config
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    dataset_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    targets = ["^BVSP"],
    train_size_ratio = 1,
    batch_size = 1,
    sequence_length = 64
    # -----------------------------------------
)

2025-07-11 20:19:00.009 | INFO     | src.features:main:36 - Generating features from dataset...
2025-07-11 20:19:00.063 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-07-11 20:19:00.092 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-07-11 20:19:00.359 | SUCCESS  | src.features:main:51 - Saving train features in C:\Repositories\time-series-prediction-rnn\data\processed...
2025-07-11 20:19:00.630 | SUCCESS  | src.features:main:55 - Saving test features in C:\Repositories\time-series-prediction-rnn\data\processed...
2025-07-11 20:19:00.633 | SUCCESS  | src.features:main:59 - Features generation complete.
2025-07-11 20:19:00.633 | INFO     | src.features:main:61 - Saving transformers...
2025-07-11 20:19:00.636 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-07-11 20:19:00.642 | INFO     | sr

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [4]:
from src import config
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy",
    # -----------------------------------------
    optimizer =  None,  # Use default optimizer
    loss = None,  # Use default loss
    metrics = None,  # Use default metrics
    # -----------------------------------------
    epochs = 2**8,    
    validation_len = 2**7,
    batch_size = 2**6,    
    # -----------------------------------------
    experiment_name = "0.0-amp-main-notebook",
)

2025-07-11 20:19:02.864 | INFO     | src.modeling.train:main:44 - Loading training dataset...
2025-07-11 20:19:02.908 | INFO     | src.modeling.train:main:52 - Selecting builder strategy...
2025-07-11 20:19:02.908 | INFO     | src.modeling.train:main:58 - Selecting compile strategy...
2025-07-11 20:19:02.908 | INFO     | src.modeling.train:main:61 - Selecting training strategy...
2025-07-11 20:19:02.908 | INFO     | src.modeling.train:main:69 - Building model training pipeline template...
2025-07-11 20:19:02.908 | INFO     | src.modeling.train:main:76 - Training model...
Epoch 1/256
36/36 ━━━━━━━━━━━━━━━━━━━━ 17s 161ms/step - loss: 0.4967 - mae: 0.8119 - mse: 20.9783 - r2_score: -0.0097 - rmse: 1.7338 - smape: 180.0077 - val_loss: 0.3576 - val_mae: 0.6506 - val_mse: 0.9616 - val_r2_score: -0.0064 - val_rmse: 0.9786 - val_smape: 189.7319 - learning_rate: 0.0010
Epoch 2/256
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - loss: 0.4929 - mae: 0.8098 - mse: 20.9666 - r2_score: -0.0069 - rmse: 1.

2025/07/11 20:25:06 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/07/11 20:25:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SAMSUNG\AppData\Local\Temp\tmputjqiz63\model, flavor: keras). Fall back to return ['keras==3.9.2']. Set logging level to DEBUG to see the full traceback. 
2025/07/11 20:25:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'regression-pipeline' already exists. Creating a new version of this model...
Created version '9' of model 'regression-pipeline'.
2025/07/11 20:25:19 INFO mlflow.pyfunc: Inferring model signature from input example


2025-07-11 20:25:19.841 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructo

2025-07-11 20:25:20.194 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


...

2025-07-11 20:25:32.836 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


2025/07/11 20:25:35 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "Close_^BVSP",
      "Volume_^BVSP",
      "Volume_^GSPC",
      "Volume_BTC-USD",
      "Volume_GC=F",
      "Volume_CL=F",
      "IPCA_Mensal",
      "IGP_M_Mensal",
      "month_name",
      "week_of_month",
      "day_of_week",
      "is_weekend",
      "is_holiday",
      "Volume_^IXIC",
      "Volume_^DJI",
      "Volume_SI=F",
      "Volume_HO=F",
      "Volume_NG=F",
      "Volume_RB=F",
      "Volume_SB=F",
      "Volume_ZC=F",
      "Close_ZR=F",
      "High_ZR=F",
      "Low_ZR=F",
      "Open_ZR=F",
      "Volume_ZR=F",
      "Volume_ZS=F",
      "Volume_KC=F",
      "Volume_LE=F",
      "Volume_HE=F",
      "Volume_CT=F",
      "Volume_ZB=F",
      "Volume_HG=F",
      "INCC_Mensal"
    ],
    "data": [
      [
        -0.0131074369774313,
        0.065989192799247,
        -0.0182436324617383,
        1.020974182190618,
        0.362488

2025-07-11 20:25:39.237 | SUCCESS  | src.utils.log.log_strategy:run:201 - MLflow logging completed.
2025-07-11 20:25:39.252 | SUCCESS  | src.modeling.train:main:107 - Experiment logged successfully.


## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [5]:
from src import config
from src.modeling import predict

predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv"
    ,preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl"
    ,postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl"
    ,output_path = config.PREDICTED_DATA_DIR / "dataset_report.csv"
    ,model_path = config.MODELS_DIR / "default_model.keras"  # Select the best model  
    # -----------------------------------------
)

2025-07-11 20:25:39.310 | INFO     | src.modeling.predict:main:27 - Performing inference for model...
2025-07-11 20:25:39.534 | INFO     | src.modeling.predict:main:42 - Input data shape: (64, 34)
2025-07-11 20:25:39.551 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
            Close_^BVSP  Volume_^BVSP       type
2025-07-09    -0.013079      0.123613       True
2025-07-10    -0.005368      0.261283       True
2025-07-11     0.002065     -0.090739  Predicted
2025-07-11 20:25:42.350 | SUCCESS  | src.modeling.predict:main:77 - Inference complete.


## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.